In [1]:
import numpy as np
import time
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.utils.rendertools import RenderTool
from flatland.envs.observations import GlobalObsForRailEnv
from flatland.envs.observations import TreeObsForRailEnv
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_env import GridTransitionMap


In [2]:
NUMBER_OF_AGENTS = 10

nr_start_goal = 10
nr_extra = 2
min_dist = 8
max_dist = 99999
# seed = 0
seed = np.random.randint(0,1000)

In [3]:
my_rail_generator = complex_rail_generator (
        nr_start_goal,
        nr_extra,
        min_dist,
        max_dist,
        seed
    )

env = RailEnv(
    
    # parameters taht control the dimensions of the map.
    width = 10,
    height = 10,
    
    # Rail generator: generates new rail networks on each reset
    # simplest rail generators: 
    #         envs.rail_generators.rail_from_manual_specifications_generator and 
    #         envs.rail_generators.random_rail_generator.
    
    rail_generator = my_rail_generator,
    schedule_generator = complex_schedule_generator(),
    obs_builder_object= GlobalObsForRailEnv(),
    number_of_agents = NUMBER_OF_AGENTS
)


In [4]:
# The original railway map from the complex_rail_generator
original_rail_map = env.rail.grid.copy()




In [5]:
# straightward implementation, could be optimized
# The node id is in the form (x,y), could be a linear mapping from 
# the most leftupper cornner to the most rightlower corner

# Flatland map encoding scheme, NESW -> N:[NESW], E:[NESW], S[NESW], W[NESW]
# Deadend cell is different, agent faces the opposite to the direction that it can goes. 


class decode_map:
    
    
    def __init__(self,original_map):
        self.original_map = original_map
        self.converted_input = {}

    # Determine next nodes for current_node.
    def case_matching(self, current_node, short_bits):
        
        next_nodes = []
        
        if (int(short_bits[0]) == 1):
            
            next_node = (current_node[0]-1, current_node[1])
#             print("N", next_node)
            
            next_nodes.append(next_node)
        
        if(int(short_bits[1]) == 1):
            
            next_node = (current_node[0], current_node[1]+1)
#             print("E", next_node)
            
            next_nodes.append(next_node)

        
        if(int(short_bits[2]) == 1):
            
            next_node = (current_node[0]+1, current_node[1])
#             print("S", next_node)
            
            next_nodes.append(next_node)

        if(int(short_bits[3]) == 1):
            
            next_node = (current_node[0], current_node[1]-1)
#             print("W", next_node)
            
            next_nodes.append(next_node)
        
        return next_nodes
        
    # First find the possible previous nodes, and call another function to find the possible next nodes.
    def slice_16_bits(self, current_node, bits):
        
        previous_nodes = {}
        
        # Facing North
        
        if (bits[0:4] != "0000"):
            
#             print(bits[0:4])
            next_nodes = self.case_matching(current_node, bits[0:4])
#             print("N, and returned next nodes", next_nodes)
            
            # add one previous node, and what nodes the agent can go from current node.
            previous_nodes[(current_node[0]+1,current_node[1])] = next_nodes
            
#             print("N, and current previous nodes", previous_nodes)
            
        # East
        if (bits[4:8] != "0000"):

#             print(bits[4:8])
            next_nodes = self.case_matching(current_node, bits[4:8])
#             print("E",next_nodes)
            
            previous_nodes[(current_node[0], current_node[1]-1)] = next_nodes
            
#             print("E, and current previous nodes", previous_nodes)
# 

        # South
        if (bits[8:12] != "0000"):
            
#             print(bits[8:12])
            next_nodes = self.case_matching(current_node, bits[8:12])
#             print("S",next_nodes)
            
            previous_nodes[(current_node[0]-1, current_node[1])] = next_nodes
            
#             print("S, and current previous nodes", previous_nodes)
       
        # West
        if (bits[12:16] != "0000"):
            
#             print(bits[12:16])
            next_nodes = self.case_matching(current_node, bits[12:16])
#             print("W",next_nodes)
            
            previous_nodes[(current_node[0], current_node[1]+1)] = next_nodes
            
#             print("W, and current previous nodes", previous_nodes)

        return previous_nodes
            
    def convert_ori_rail_map(self):
        for row in range(0,len(self.original_map)):
            for col in range(0,len(self.original_map[row])):
                
                # Current node: (row,col)
                print("Current node:", (row,col))
                current_node = (row,col)
                # Get previous node, decided by which direction the agent is facing.
                # EXCEPT the deadend nodes.
                
                
#                 print('{0:016b}'.format(self.original_map[row][col]))
                previous_next_nodes = self.slice_16_bits(current_node, '{0:016b}'.format(self.original_map[row][col]))
#                 print(previous_next_nodes.items())
                
                for previous,direction in previous_next_nodes.items():
#                     print("direction", direction)

                    self.converted_input[(current_node, previous)] = direction
#         print(self.converted_input)
        return self.converted_input

In [6]:
result = decode_map(original_rail_map).convert_ori_rail_map()

Current node: (0, 0)
Current node: (0, 1)
Current node: (0, 2)
Current node: (0, 3)
Current node: (0, 4)
Current node: (0, 5)
Current node: (0, 6)
Current node: (0, 7)
Current node: (0, 8)
Current node: (0, 9)
Current node: (1, 0)
Current node: (1, 1)
Current node: (1, 2)
Current node: (1, 3)
Current node: (1, 4)
Current node: (1, 5)
Current node: (1, 6)
Current node: (1, 7)
Current node: (1, 8)
Current node: (1, 9)
Current node: (2, 0)
Current node: (2, 1)
Current node: (2, 2)
Current node: (2, 3)
Current node: (2, 4)
Current node: (2, 5)
Current node: (2, 6)
Current node: (2, 7)
Current node: (2, 8)
Current node: (2, 9)
Current node: (3, 0)
Current node: (3, 1)
Current node: (3, 2)
Current node: (3, 3)
Current node: (3, 4)
Current node: (3, 5)
Current node: (3, 6)
Current node: (3, 7)
Current node: (3, 8)
Current node: (3, 9)
Current node: (4, 0)
Current node: (4, 1)
Current node: (4, 2)
Current node: (4, 3)
Current node: (4, 4)
Current node: (4, 5)
Current node: (4, 6)
Current node:

In [7]:
# pretty print result

for i in result.items():
    print(i,"\n")

(((0, 0), (0, 1)), [(0, 1)]) 

(((0, 1), (0, 0)), [(0, 2)]) 

(((0, 1), (0, 2)), [(0, 0)]) 

(((0, 2), (1, 2)), [(0, 1)]) 

(((0, 2), (0, 1)), [(1, 2)]) 

(((0, 4), (1, 4)), [(1, 4)]) 

(((0, 6), (1, 6)), [(0, 7)]) 

(((0, 6), (0, 7)), [(1, 6)]) 

(((0, 7), (0, 6)), [(0, 8)]) 

(((0, 7), (0, 8)), [(0, 6)]) 

(((0, 8), (0, 7)), [(0, 7)]) 

(((1, 1), (2, 1)), [(2, 1)]) 

(((1, 2), (2, 2)), [(1, 3)]) 

(((1, 2), (0, 2)), [(1, 3)]) 

(((1, 2), (1, 3)), [(0, 2), (2, 2)]) 

(((1, 3), (2, 3)), [(1, 4)]) 

(((1, 3), (1, 2)), [(1, 4)]) 

(((1, 3), (1, 4)), [(2, 3), (1, 2)]) 

(((1, 4), (2, 4)), [(0, 4), (1, 5)]) 

(((1, 4), (1, 3)), [(1, 5)]) 

(((1, 4), (0, 4)), [(2, 4)]) 

(((1, 4), (1, 5)), [(2, 4), (1, 3)]) 

(((1, 5), (2, 5)), [(1, 4)]) 

(((1, 5), (1, 4)), [(1, 6), (2, 5)]) 

(((1, 5), (1, 6)), [(1, 4)]) 

(((1, 6), (1, 5)), [(0, 6), (1, 7)]) 

(((1, 6), (0, 6)), [(1, 5)]) 

(((1, 6), (1, 7)), [(1, 5)]) 

(((1, 7), (1, 6)), [(1, 8)]) 

(((1, 7), (1, 8)), [(1, 6)]) 

(((1, 8), (1, 7)), [(1

In [8]:
# Obtain the start and target locations
# Assuming that each agent is going from one assigned start location to one fixed target location.
# i.e. no multiple targets available for one agent to go to.

# can be access by env.agents (lists of agents objects with their own information)
'''
informaiton include: poistion (i.e. start location at time step 0), 
                     direction,
                     target,
                     speed_data,
                     malfunction_data,
                     handle,
                     old_direction,
                     old_position     
'''

agent_info = {}

for index, agent in enumerate(env.agents):
    print(index, agent)
   

0 EnvAgent(position=(3, 5), direction=0, target=(9, 8), moving=False, speed_data={'position_fraction': 0.0, 'speed': 1.0, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, handle=0, old_direction=None, old_position=None)
1 EnvAgent(position=(8, 2), direction=3, target=(0, 4), moving=False, speed_data={'position_fraction': 0.0, 'speed': 1.0, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, handle=1, old_direction=None, old_position=None)
2 EnvAgent(position=(5, 5), direction=1, target=(1, 1), moving=False, speed_data={'position_fraction': 0.0, 'speed': 1.0, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, handle=2, old_direction=None, old_position=None)
3 EnvAgent(position=(2, 7), direction=1, target=(4, 1),

In [8]:
# tool to render environments
env_renderer = RenderTool(env, gl="PIL")
# env_renderer = RenderTool(env)

# env_renderer = RenderTool (env)
env_renderer.render_env(show=True)


C:\Users\yizh9\Anaconda3\envs\flatland-rl\lib\site-packages\flatland\utils\rendertools.py:289: UserWarning: Predictor did not provide any predicted cells to render.                 Observation builder needs to populate: env.dev_obs_dict
  Observation builder needs to populate: env.dev_obs_dict")


In [ ]:
'{0:016b}'.format(33825)

In [10]:
idx2node = {idx:k for idx, k in enumerate(result.keys())} 
node2idx = {k:idx for idx, k in idx2node.items()}
idx2pos = {k:v[0] for (k,v) in idx2node.items()}

edges = []
for cur, prev in result:
    print (prev, cur, result[(cur, prev)])
    for to in result[(cur, prev)]:
        print(to)
        edges.append((node2idx[(cur, prev)], node2idx[(to, cur)]))

(0, 1) (0, 0) [(0, 1)]
(0, 1)
(0, 0) (0, 1) [(0, 2)]
(0, 2)
(0, 2) (0, 1) [(0, 0)]
(0, 0)
(1, 2) (0, 2) [(0, 1)]
(0, 1)
(0, 1) (0, 2) [(1, 2)]
(1, 2)
(1, 4) (0, 4) [(1, 4)]
(1, 4)
(1, 6) (0, 6) [(0, 7)]
(0, 7)
(0, 7) (0, 6) [(1, 6)]
(1, 6)
(0, 6) (0, 7) [(0, 8)]
(0, 8)
(0, 8) (0, 7) [(0, 6)]
(0, 6)
(0, 7) (0, 8) [(0, 7)]
(0, 7)
(2, 1) (1, 1) [(2, 1)]
(2, 1)
(2, 2) (1, 2) [(1, 3)]
(1, 3)
(0, 2) (1, 2) [(1, 3)]
(1, 3)
(1, 3) (1, 2) [(0, 2), (2, 2)]
(0, 2)
(2, 2)
(2, 3) (1, 3) [(1, 4)]
(1, 4)
(1, 2) (1, 3) [(1, 4)]
(1, 4)
(1, 4) (1, 3) [(2, 3), (1, 2)]
(2, 3)
(1, 2)
(2, 4) (1, 4) [(0, 4), (1, 5)]
(0, 4)
(1, 5)
(1, 3) (1, 4) [(1, 5)]
(1, 5)
(0, 4) (1, 4) [(2, 4)]
(2, 4)
(1, 5) (1, 4) [(2, 4), (1, 3)]
(2, 4)
(1, 3)
(2, 5) (1, 5) [(1, 4)]
(1, 4)
(1, 4) (1, 5) [(1, 6), (2, 5)]
(1, 6)
(2, 5)
(1, 6) (1, 5) [(1, 4)]
(1, 4)
(1, 5) (1, 6) [(0, 6), (1, 7)]
(0, 6)
(1, 7)
(0, 6) (1, 6) [(1, 5)]
(1, 5)
(1, 7) (1, 6) [(1, 5)]
(1, 5)
(1, 6) (1, 7) [(1, 8)]
(1, 8)
(1, 8) (1, 7) [(1, 6)]
(1, 6)
(1, 7) (1,

In [11]:
idx2node

{0: ((0, 0), (0, 1)),
 1: ((0, 1), (0, 0)),
 2: ((0, 1), (0, 2)),
 3: ((0, 2), (1, 2)),
 4: ((0, 2), (0, 1)),
 5: ((0, 4), (1, 4)),
 6: ((0, 6), (1, 6)),
 7: ((0, 6), (0, 7)),
 8: ((0, 7), (0, 6)),
 9: ((0, 7), (0, 8)),
 10: ((0, 8), (0, 7)),
 11: ((1, 1), (2, 1)),
 12: ((1, 2), (2, 2)),
 13: ((1, 2), (0, 2)),
 14: ((1, 2), (1, 3)),
 15: ((1, 3), (2, 3)),
 16: ((1, 3), (1, 2)),
 17: ((1, 3), (1, 4)),
 18: ((1, 4), (2, 4)),
 19: ((1, 4), (1, 3)),
 20: ((1, 4), (0, 4)),
 21: ((1, 4), (1, 5)),
 22: ((1, 5), (2, 5)),
 23: ((1, 5), (1, 4)),
 24: ((1, 5), (1, 6)),
 25: ((1, 6), (1, 5)),
 26: ((1, 6), (0, 6)),
 27: ((1, 6), (1, 7)),
 28: ((1, 7), (1, 6)),
 29: ((1, 7), (1, 8)),
 30: ((1, 8), (1, 7)),
 31: ((1, 8), (1, 9)),
 32: ((1, 9), (1, 8)),
 33: ((2, 1), (3, 1)),
 34: ((2, 1), (1, 1)),
 35: ((2, 2), (1, 2)),
 36: ((2, 3), (3, 3)),
 37: ((2, 3), (1, 3)),
 38: ((2, 4), (3, 4)),
 39: ((2, 4), (1, 4)),
 40: ((2, 5), (1, 5)),
 41: ((2, 5), (2, 6)),
 42: ((2, 6), (3, 6)),
 43: ((2, 6), (2, 5))

In [12]:
node2idx

{((0, 0), (0, 1)): 0,
 ((0, 1), (0, 0)): 1,
 ((0, 1), (0, 2)): 2,
 ((0, 2), (1, 2)): 3,
 ((0, 2), (0, 1)): 4,
 ((0, 4), (1, 4)): 5,
 ((0, 6), (1, 6)): 6,
 ((0, 6), (0, 7)): 7,
 ((0, 7), (0, 6)): 8,
 ((0, 7), (0, 8)): 9,
 ((0, 8), (0, 7)): 10,
 ((1, 1), (2, 1)): 11,
 ((1, 2), (2, 2)): 12,
 ((1, 2), (0, 2)): 13,
 ((1, 2), (1, 3)): 14,
 ((1, 3), (2, 3)): 15,
 ((1, 3), (1, 2)): 16,
 ((1, 3), (1, 4)): 17,
 ((1, 4), (2, 4)): 18,
 ((1, 4), (1, 3)): 19,
 ((1, 4), (0, 4)): 20,
 ((1, 4), (1, 5)): 21,
 ((1, 5), (2, 5)): 22,
 ((1, 5), (1, 4)): 23,
 ((1, 5), (1, 6)): 24,
 ((1, 6), (1, 5)): 25,
 ((1, 6), (0, 6)): 26,
 ((1, 6), (1, 7)): 27,
 ((1, 7), (1, 6)): 28,
 ((1, 7), (1, 8)): 29,
 ((1, 8), (1, 7)): 30,
 ((1, 8), (1, 9)): 31,
 ((1, 9), (1, 8)): 32,
 ((2, 1), (3, 1)): 33,
 ((2, 1), (1, 1)): 34,
 ((2, 2), (1, 2)): 35,
 ((2, 3), (3, 3)): 36,
 ((2, 3), (1, 3)): 37,
 ((2, 4), (3, 4)): 38,
 ((2, 4), (1, 4)): 39,
 ((2, 5), (1, 5)): 40,
 ((2, 5), (2, 6)): 41,
 ((2, 6), (3, 6)): 42,
 ((2, 6), (2, 5)): 43

In [13]:
idx2pos

{0: (0, 0),
 1: (0, 1),
 2: (0, 1),
 3: (0, 2),
 4: (0, 2),
 5: (0, 4),
 6: (0, 6),
 7: (0, 6),
 8: (0, 7),
 9: (0, 7),
 10: (0, 8),
 11: (1, 1),
 12: (1, 2),
 13: (1, 2),
 14: (1, 2),
 15: (1, 3),
 16: (1, 3),
 17: (1, 3),
 18: (1, 4),
 19: (1, 4),
 20: (1, 4),
 21: (1, 4),
 22: (1, 5),
 23: (1, 5),
 24: (1, 5),
 25: (1, 6),
 26: (1, 6),
 27: (1, 6),
 28: (1, 7),
 29: (1, 7),
 30: (1, 8),
 31: (1, 8),
 32: (1, 9),
 33: (2, 1),
 34: (2, 1),
 35: (2, 2),
 36: (2, 3),
 37: (2, 3),
 38: (2, 4),
 39: (2, 4),
 40: (2, 5),
 41: (2, 5),
 42: (2, 6),
 43: (2, 6),
 44: (2, 6),
 45: (2, 7),
 46: (3, 1),
 47: (3, 1),
 48: (3, 2),
 49: (3, 2),
 50: (3, 3),
 51: (3, 3),
 52: (3, 3),
 53: (3, 4),
 54: (3, 4),
 55: (3, 5),
 56: (3, 6),
 57: (3, 6),
 58: (3, 6),
 59: (3, 7),
 60: (3, 7),
 61: (3, 8),
 62: (3, 8),
 63: (4, 1),
 64: (4, 2),
 65: (4, 2),
 66: (4, 3),
 67: (4, 3),
 68: (4, 3),
 69: (4, 3),
 70: (4, 4),
 71: (4, 4),
 72: (4, 4),
 73: (4, 4),
 74: (4, 5),
 75: (4, 5),
 76: (4, 5),
 77: (4, 

In [ ]:
import os
nowtime = os.popen('g++ ./main.cpp')
result = os.popen('./a.out')
print (result.read())

In [ ]:
import threading

def work1():
    os.popen('g++ ./main.cpp -o a.out')
    result = os.popen('./a.out')
    print(result.read())
def work2():
    os.popen('g++ ./main.cpp -o a1.out')
    result = os.popen('./a1.out')
    print(result.read())
    return result 
    
t1 = threading.Thread(target=work1)
t2 = threading.Thread(target=work2)

t1.start()
t2.start()

In [ ]:
import threading
 
def work1: 
    for i in range(x,y):
        print(i)
 
ta = threading.Thread(target=threadfun,args=(1,6))      
tb = threading.Thread(target=threadfun,args=(10,15))    
ta.start()          


In [9]:
def my_controller():
    """
    You are supposed to write this controller
    """
    _action = {}
    for _idx in range(NUMBER_OF_AGENTS):
        _action[_idx] = np.random.randint(0, 5)
    return _action

for step in range(1):

    # Get agents' handles to give actions
    # handles = env.get_agent_handles()
    
    # e.g. giving two agents actions. 
    # action_dict = {handles[0]:0, handles[1]:0}
    
    _action = my_controller()
    
    # obs, all_rewards, done: dictionary indexed by agents handles
        # values: correspond to the relevant obeservations, rewards and terminal 
        #       status for each agent.
        
    obs, all_rewards, done, _ = env.step(_action) # environment take one step with the provided actions.
    
    # show results
    print("Observation: \n", obs)
    print("Rewards: {}, [done={}]".format( all_rewards, done))
    env_renderer.render_env(show=True, frames=False, show_observations=False)
    time.sleep(1)

Observation: 
 {0: (array([[[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 1., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1.,

NameError: name 'env_renderer' is not defined